In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys;
sys.path.append(r'C:\Monitor\\')
sys.path.append(r'\\ac-geneva-24\E\grains trading\Streamlit\Monitor\\')

from datetime import datetime as dt
from copy import deepcopy

import pandas as pd; pd.options.mode.chained_assignment = None
import Utilities.Modeling as um
import Utilities.Weather as uw
import Utilities.GLOBAL as GV
import Utilities.Modeling as um
import Models.Corn_USA_Yield as cy

import plotly.express as px

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
prec_units='in'
temp_units='F'

scope = cy.Define_Scope()
raw_data = cy.Get_Data_All_Parallel(scope)
raw_data.keys()

dict_keys(['years', 'locations', 'yield', 'weights', 'planting_progress', 'silking_progress', 'w_df_all', 'w_w_df_all'])

In [4]:
milestones =cy.Milestone_from_Progress(raw_data)
intervals = cy.Intervals_from_Milestones(milestones)

train_DF_instr = um.Build_DF_Instructions('weighted',GV.WD_HIST, prec_units=prec_units, temp_units=temp_units, ext_mode = GV.EXT_DICT)
train_df = cy.Build_DF(raw_data, milestones, intervals, train_DF_instr)

model = um.Fit_Model(train_df,'Yield',GV.CUR_YEAR)
# print(model.summary())

In [5]:
date_start=dt(2022,4,1)
date_end=dt(2022,9,30)
ext_dict={'Prec' :'Mean','Sdd30':'Mean'}
pred_DF_instr=um.Build_DF_Instructions('weighted',GV.WD_H_GFS, prec_units='in', temp_units='F',ext_mode = ext_dict)
pred_DF = cy.Build_Pred_DF(raw_data, milestones, pred_DF_instr,date_start=date_start, date_end=date_end)
yields=model.predict(pred_DF[model.params.index])
pred_DF['Yield']=yields
pred_DF

,const,Trend,Yield,Planted pct on May 15th,Planting Prec,Planting Prec Squared,Jul Aug Prec,Jul Aug Prec Squared,Prec Interaction,Pollination SDD,Regular SDD
2022-04-01,1.0,2022,180.691389,72.289575,9.033594,81.605824,9.889243,97.797123,89.335406,60.621859,63.089836
2022-04-02,1.0,2022,180.691389,72.289575,9.033594,81.605824,9.889243,97.797123,89.335406,60.621859,63.089836
2022-04-03,1.0,2022,180.691389,72.289575,9.033594,81.605824,9.889243,97.797123,89.335406,60.621859,63.089836
2022-04-04,1.0,2022,180.691389,72.289575,9.033594,81.605824,9.889243,97.797123,89.335406,60.621859,63.089836
2022-04-05,1.0,2022,180.691389,72.289575,9.033594,81.605824,9.889243,97.797123,89.335406,60.621859,63.089836
...,...,...,...,...,...,...,...,...,...,...,...
2022-09-26,1.0,2022,172.303562,49.000000,8.351362,69.745245,8.896639,79.150181,74.299049,62.893021,82.281449
2022-09-27,1.0,2022,172.303562,49.000000,8.351362,69.745245,8.896639,79.150181,74.299049,62.893021,82.281449
2022-09-28,1.0,2022,172.303562,49.000000,8.351362,69.745245,8.896639,79.150181,74.299049,62.893021,82.281449
2022-09-29,1.0,2022,172.303562,49.000000,8.351362,69.745245,8.896639,79.150181,74.299049,62.893021,82.281449


In [6]:
date_start=dt(2022,4,1)
date_end=dt(2022,9,30)
ext_dict={'Prec' :'Mean','Sdd30':'Mean'}
pred_DF_instr=um.Build_DF_Instructions('weighted',GV.WD_H_GFS, prec_units='in', temp_units='F',ext_mode = ext_dict)
int_pred_DF = cy.Build_Intuitive_Pred_DF(raw_data, milestones, pred_DF_instr,date_start=date_start, date_end=date_end)
yields=model.predict(int_pred_DF[model.params.index])
int_pred_DF['Yield']=yields
int_pred_DF

,const,Trend,Yield,Planted pct on May 15th,Planting Prec,Planting Prec Squared,Jul Aug Prec,Jul Aug Prec Squared,Prec Interaction,Pollination SDD,Regular SDD
2022-04-01,1.0,2022,33.241679,72.289575,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-04-02,1.0,2022,33.241679,72.289575,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-04-03,1.0,2022,33.241679,72.289575,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-04-04,1.0,2022,33.241679,72.289575,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-04-05,1.0,2022,33.241679,72.289575,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
2022-09-26,1.0,2022,172.303562,49.000000,8.351362,69.745245,8.896639,79.150181,74.299049,62.893021,82.281449
2022-09-27,1.0,2022,172.303562,49.000000,8.351362,69.745245,8.896639,79.150181,74.299049,62.893021,82.281449
2022-09-28,1.0,2022,172.303562,49.000000,8.351362,69.745245,8.896639,79.150181,74.299049,62.893021,82.281449
2022-09-29,1.0,2022,172.303562,49.000000,8.351362,69.745245,8.896639,79.150181,74.299049,62.893021,82.281449


In [7]:
date_start=dt(2022,4,1)
date_end=dt(2022,9,30)
ext_dict={'Prec' :'Mean','Sdd30':'Analog_2012'}
pred_DF_instr=um.Build_DF_Instructions('weighted',GV.WD_H_GFS, prec_units='in', temp_units='F',ext_mode = ext_dict)
int_pred_DF_2012 = cy.Build_Intuitive_Pred_DF(raw_data, milestones, pred_DF_instr,date_start=date_start, date_end=date_end)
yields=model.predict(int_pred_DF_2012[model.params.index])
int_pred_DF_2012['Yield']=yields
int_pred_DF_2012

,const,Trend,Yield,Planted pct on May 15th,Planting Prec,Planting Prec Squared,Jul Aug Prec,Jul Aug Prec Squared,Prec Interaction,Pollination SDD,Regular SDD
2022-04-01,1.0,2022,33.241679,72.289575,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-04-02,1.0,2022,33.241679,72.289575,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-04-03,1.0,2022,33.241679,72.289575,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-04-04,1.0,2022,33.241679,72.289575,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-04-05,1.0,2022,33.241679,72.289575,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
2022-09-26,1.0,2022,158.935545,49.000000,8.351362,69.745245,8.896639,79.150181,74.299049,165.440523,115.718879
2022-09-27,1.0,2022,158.935545,49.000000,8.351362,69.745245,8.896639,79.150181,74.299049,165.440523,115.718879
2022-09-28,1.0,2022,158.935545,49.000000,8.351362,69.745245,8.896639,79.150181,74.299049,165.440523,115.718879
2022-09-29,1.0,2022,158.935545,49.000000,8.351362,69.745245,8.896639,79.150181,74.299049,165.440523,115.718879


In [12]:
pred_chart=pd.concat([int_pred_DF['Yield'],int_pred_DF_2012['Yield']],axis=1,keys=['Base','2012'])
fig=px.line(pred_chart); fig.show('browser')

d_yield=int_pred_DF_2012['Yield']-int_pred_DF['Yield']
fig=px.line(d_yield); fig.show('browser')

d_yield=pred_DF['Yield']
fig=px.line(d_yield); fig.show('browser')

d_yield_2022=pred_DF['Yield']
d_yield_2012=pred_DF['Yield']+int_pred_DF_2012['Yield']-int_pred_DF['Yield']

final_chart=pd.concat([d_yield_2022,d_yield_2012],axis=1,keys=['2022','2012'])
fig=px.line(final_chart); fig.show('browser')

In [ ]:
pred_chart=pd.concat([pred_DF_2012['Yield'],pred_DF['Yield']],axis=1,keys=['2012','Base'])
base_chart=pd.concat([pred_DF_2012['Yield'],pred_DF['Yield']],axis=1,keys=['2012','Base'])
d_pred_chart=pd.concat([d_pred_DF_2012['Yield'],d_pred_DF['Yield']],axis=1,keys=['2012','Base'])


fig=px.line(pred_DF['Yield']-pred_DF_2012['Yield']); fig.show()
fig=px.line(pred_chart); fig.show()
fig=px.line(d_pred_chart); fig.show()
# px.line(df_study)

# px.line(df_study)
